In [2]:
import pandas as pd
import numpy as np
import datetime

%run ./classifica_serie_a.ipynb

ERROR:root:File `'./classifica_serie_a.ipynb.py'` not found.


In [10]:
cal = pd.read_csv('./datasets/calendario_completo_processed.csv')
cal.loc[cal['casa_prov']=='NP','casa_prov']='NA'
cal.loc[cal['trasferta_prov']=='NP','trasferta_prov']='NA'
cal.head()

,data,diretta,casa,trasferta,casa_prov,trasferta_prov
0,2019-10-29 19:00:00,DAZN,Parma,Hellas Verona,PR,VR
1,2019-10-29 21:00:00,SKY,Brescia,Inter,BS,MI
2,2019-10-30 19:00:00,SKY,Napoli,Atalanta,NA,BG
3,2019-10-30 21:00:00,SKY,Cagliari,Bologna,CA,BO
4,2019-10-30 21:00:00,SKY,Juventus,Genoa,TO,GE


In [11]:
cal.data.unique()

array(['2019-10-29 19:00:00', '2019-10-29 21:00:00',
       '2019-10-30 19:00:00', '2019-10-30 21:00:00',
       '2019-10-31 21:00:00', '2019-11-02 15:00:00',
       '2019-11-02 18:00:00', '2019-11-02 20:45:00',
       '2019-11-03 12:30:00', '2019-11-03 15:00:00',
       '2019-11-03 18:00:00', '2019-11-03 20:45:00',
       '2019-11-04 20:45:00', '2019-11-08 20:45:00',
       '2019-11-09 15:00:00', '2019-11-09 18:00:00',
       '2019-11-09 20:45:00', '2019-11-10 12:30:00',
       '2019-11-10 15:00:00', '2019-11-10 18:00:00',
       '2019-11-10 20:45:00', '2019-11-23 15:00:00',
       '2019-11-23 18:00:00', '2019-11-23 20:45:00',
       '2019-11-24 12:30:00', '2019-11-24 15:00:00',
       '2019-11-24 18:00:00', '2019-11-25 15:00:00',
       '2019-11-25 20:45:00', '2019-11-30 15:00:00',
       '2019-11-30 18:00:00', '2019-11-30 20:45:00',
       '2019-12-01 12:30:00', '2019-12-01 15:00:00',
       '2019-12-01 18:00:00', '2019-12-01 20:45:00',
       '2019-12-02 20:45:00', '2019-12-06 20:4

In [12]:
def set_game_flag(team1, team2, s_d, squadra):
    value = 0
    if team1==squadra or team2==squadra:
        value = 0.25 if s_d=='SKY' else 1
    return value

def calculate_match_vector(df):
    match = pd.DataFrame(columns=['DATA', 'GAME_J', 'GAME_I', 'GAME_M', 'GAME_R', 'GAME_N'])
    
    for index, row in df.iterrows():
        if row['data'] not in match['DATA'].unique():
            match = match.append(pd.Series([row['data'], 0, 0, 0, 0, 0], index=match.columns ), ignore_index=True)

        match.loc[(match['DATA']==row['data']) & (match['GAME_J']==0),'GAME_J']=set_game_flag(
            row['casa'], row['trasferta'] , row['diretta'], 'Juventus')
        match.loc[(match['DATA']==row['data']) & (match['GAME_I']==0),'GAME_I']=set_game_flag(
            row['casa'], row['trasferta'] , row['diretta'], 'Inter')
        match.loc[(match['DATA']==row['data']) & (match['GAME_M']==0),'GAME_M']=set_game_flag(
            row['casa'], row['trasferta'] , row['diretta'], 'Milan')
        match.loc[(match['DATA']==row['data']) & (match['GAME_R']==0),'GAME_R']=set_game_flag(
            row['casa'], row['trasferta'] , row['diretta'], 'Roma')
        match.loc[(match['DATA']==row['data']) & (match['GAME_N']==0),'GAME_N']=set_game_flag(
            row['casa'], row['trasferta'] , row['diretta'], 'Napoli')
    return match

match=calculate_match_vector(cal)
match

,DATA,GAME_J,GAME_I,GAME_M,GAME_R,GAME_N
0,2019-10-29 19:00:00,0,0,0,0,0
1,2019-10-29 21:00:00,0,0.25,0,0,0
2,2019-10-30 19:00:00,0,0,0,0,0.25
3,2019-10-30 21:00:00,0.25,0,0,1,0
4,2019-10-31 21:00:00,0,0,0.25,0,0
...,...,...,...,...,...,...
176,2019-10-26 20:45:00,0,0,0,0,0
177,2019-10-27 12:30:00,0,0,0,0,0
178,2019-10-27 15:00:00,0,0,0,0,1
179,2019-10-27 18:00:00,0,0,0.25,0.25,0


In [13]:
match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   DATA    181 non-null    object
 1   GAME_J  181 non-null    object
 2   GAME_I  181 non-null    object
 3   GAME_M  181 non-null    object
 4   GAME_R  181 non-null    object
 5   GAME_N  181 non-null    object
dtypes: object(6)
memory usage: 8.6+ KB


In [14]:
#Anagrafica prelevata da https://www.istat.it/it/archivio/6789
prv = pd.read_csv('./datasets/Elenco-comuni-italiani.csv', sep=';', encoding='latin-1', keep_default_na=False, na_values=['_'])
prv

,Codice Regione,Codice dell'Unità territoriale sovracomunale \n(valida a fini statistici),Codice Provincia (Storico)(1),Progressivo del Comune (2),Codice Comune formato alfanumerico,Denominazione (Italiana e straniera),Denominazione in italiano,Denominazione altra lingua,Codice Ripartizione Geografica,Ripartizione geografica,...,Sigla automobilistica,Codice Comune formato numerico,Codice Comune numerico con 110 province (dal 2010 al 2016),Codice Comune numerico con 107 province (dal 2006 al 2009),Codice Comune numerico con 103 province (dal 1995 al 2005),Codice Catastale del comune,Popolazione legale 2011 (09/10/2011),NUTS1,NUTS2(3),NUTS3
0,1,201,1,1,1001,Agliè,Agliè,,1,Nord-ovest,...,TO,1001,1001,1001,1001,A074,2.644,ITC,ITC1,ITC11
1,1,201,1,2,1002,Airasca,Airasca,,1,Nord-ovest,...,TO,1002,1002,1002,1002,A109,3.819,ITC,ITC1,ITC11
2,1,201,1,3,1003,Ala di Stura,Ala di Stura,,1,Nord-ovest,...,TO,1003,1003,1003,1003,A117,462,ITC,ITC1,ITC11
3,1,201,1,4,1004,Albiano d'Ivrea,Albiano d'Ivrea,,1,Nord-ovest,...,TO,1004,1004,1004,1004,A157,1.791,ITC,ITC1,ITC11
4,1,201,1,6,1006,Almese,Almese,,1,Nord-ovest,...,TO,1006,1006,1006,1006,A218,6.303,ITC,ITC1,ITC11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7899,20,111,111,103,111103,Villaputzu,Villaputzu,,5,Isole,...,SU,111103,92097,92097,92097,L998,4.836,ITG,ITG2,ITG27
7900,20,111,111,104,111104,Villasalto,Villasalto,,5,Isole,...,SU,111104,92098,92098,92098,M016,1.127,ITG,ITG2,ITG27
7901,20,111,111,105,111105,Villasimius,Villasimius,,5,Isole,...,SU,111105,92100,92100,92100,B738,3.420,ITG,ITG2,ITG27
7902,20,111,111,106,111106,Villasor,Villasor,,5,Isole,...,SU,111106,92101,92101,92101,M025,6.857,ITG,ITG2,ITG27


In [15]:
prv=prv.drop_duplicates(['Denominazione regione', 'Sigla automobilistica'])[
    ['Denominazione regione', 'Sigla automobilistica']]
prv.rename(columns={'Denominazione regione':'REGIONE', 
                    'Sigla automobilistica':'PROVINCIA'
                   }, 
                 inplace=True)

# List of series  
listOfSeries = [pd.Series(['Sardegna', 'OT'], index=prv.columns ) ,
                pd.Series(['Sardegna', 'OG'], index=prv.columns ) ,
                pd.Series(['Sardegna', 'CI'], index=prv.columns ),
                pd.Series(['Emilia-Romagna', 'SM'], index=prv.columns ), #Repubblica di San Marino inserita per comodità in Emilia
                pd.Series(['Sardegna', 'VS'], index=prv.columns )  ]
prv = prv.append(listOfSeries , ignore_index=True)
prv.reset_index(drop=True)

,REGIONE,PROVINCIA
0,Piemonte,TO
1,Piemonte,VC
2,Piemonte,NO
3,Piemonte,CN
4,Piemonte,AT
...,...,...
107,Sardegna,OT
108,Sardegna,OG
109,Sardegna,CI
110,Emilia-Romagna,SM


In [16]:
prv[prv['PROVINCIA']=='NA']

,REGIONE,PROVINCIA
77,Campania,NA


In [17]:
#Anagrafica basata sui dati https://forum.termometropolitico.it/170882-percentuale-di-tifosi-per-squadra-italia-regione-per-regione.html
# create a dictionary 
dictionary = {'REGIONE' : pd.Series(['Piemonte', 'Lombardia', "Valle d'Aosta/Vallée d'Aoste", 'Trentino-Alto Adige/Südtirol', 'Veneto', 'Friuli-Venezia Giulia', 'Liguria', 'Emilia-Romagna', 'Toscana', 'Umbria', 'Marche', 'Lazio', 'Abruzzo', 'Molise', 'Campania', 'Puglia', 'Basilicata', 'Calabria', 'Sicilia', 'Sardegna']),
              'JUVENTUS' : pd.Series([34.1, 22.5, 34.1, 33.4, 28.1, 24.9, 20.4, 24.2, 23.2, 23, 34, 16.3, 33.5, 32.9, 17.1, 28.8, 31.3, 29.6, 33.8, 23.4]),
              'INTER' : pd.Series([11.2, 26.9, 11.2, 17.1, 16.9, 12.8, 9.2, 15, 12.4, 16.1, 17.9, 6, 19.8, 19, 10, 14.4, 26, 16.4, 17.2, 13.2] ),
              'MILAN' : pd.Series([26.1, 10.9, 26.1, 24.3, 22.5, 23.5, 10.5, 18.7, 16.5, 18, 18.4, 6.5, 19.2, 22.2, 9.4, 19.9, 15.1, 14.1, 19.1, 12.6]), 
              'ROMA' : pd.Series([1.5, np.NaN, 1.5, 3.9, np.NaN, np.NaN, 1.7, np.NaN, 2.5, 4.4, 3.3, 35.1, 5.8, 4.2, 2.9, 4.4, 3.8, 4.4, 3.8, 3.5]),
              'NAPOLI' : pd.Series([3.3, 4.6, 3.3, 4.5, 3.6, 2.8, 3.4, 3.3, 4.5, np.NaN, np.NaN, 4, 3.2, 7.9, 46.7, 4.7, 8.6, 4.5, 3.2, 2.7])}

anag_tifosi = pd.DataFrame(dictionary)
anag_tifosi=anag_tifosi.fillna(0)
anag_tifosi

,REGIONE,JUVENTUS,INTER,MILAN,ROMA,NAPOLI
0,Piemonte,34.1,11.2,26.1,1.5,3.3
1,Lombardia,22.5,26.9,10.9,0.0,4.6
2,Valle d'Aosta/Vallée d'Aoste,34.1,11.2,26.1,1.5,3.3
3,Trentino-Alto Adige/Südtirol,33.4,17.1,24.3,3.9,4.5
4,Veneto,28.1,16.9,22.5,0.0,3.6
5,Friuli-Venezia Giulia,24.9,12.8,23.5,0.0,2.8
6,Liguria,20.4,9.2,10.5,1.7,3.4
7,Emilia-Romagna,24.2,15.0,18.7,0.0,3.3
8,Toscana,23.2,12.4,16.5,2.5,4.5
9,Umbria,23.0,16.1,18.0,4.4,0.0


In [18]:
anag_tifosi.JUVENTUS=anag_tifosi.JUVENTUS/100
anag_tifosi.INTER=anag_tifosi.INTER/100
anag_tifosi.MILAN=anag_tifosi.MILAN/100
anag_tifosi.ROMA=anag_tifosi.ROMA/100
anag_tifosi.NAPOLI=anag_tifosi.NAPOLI/100
anag_tifosi

,REGIONE,JUVENTUS,INTER,MILAN,ROMA,NAPOLI
0,Piemonte,0.341,0.112,0.261,0.015,0.033
1,Lombardia,0.225,0.269,0.109,0.000,0.046
2,Valle d'Aosta/Vallée d'Aoste,0.341,0.112,0.261,0.015,0.033
3,Trentino-Alto Adige/Südtirol,0.334,0.171,0.243,0.039,0.045
4,Veneto,0.281,0.169,0.225,0.000,0.036
5,Friuli-Venezia Giulia,0.249,0.128,0.235,0.000,0.028
6,Liguria,0.204,0.092,0.105,0.017,0.034
7,Emilia-Romagna,0.242,0.150,0.187,0.000,0.033
8,Toscana,0.232,0.124,0.165,0.025,0.045
9,Umbria,0.230,0.161,0.180,0.044,0.000


In [19]:
anag_tifosi['key']=1
match['key']=1
df_anag = pd.merge(anag_tifosi, match, on='key')
df_anag

,REGIONE,JUVENTUS,INTER,MILAN,ROMA,NAPOLI,key,DATA,GAME_J,GAME_I,GAME_M,GAME_R,GAME_N
0,Piemonte,0.341,0.112,0.261,0.015,0.033,1,2019-10-29 19:00:00,0,0,0,0,0
1,Piemonte,0.341,0.112,0.261,0.015,0.033,1,2019-10-29 21:00:00,0,0.25,0,0,0
2,Piemonte,0.341,0.112,0.261,0.015,0.033,1,2019-10-30 19:00:00,0,0,0,0,0.25
3,Piemonte,0.341,0.112,0.261,0.015,0.033,1,2019-10-30 21:00:00,0.25,0,0,1,0
4,Piemonte,0.341,0.112,0.261,0.015,0.033,1,2019-10-31 21:00:00,0,0,0.25,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,Sardegna,0.234,0.132,0.126,0.035,0.027,1,2019-10-26 20:45:00,0,0,0,0,0
3616,Sardegna,0.234,0.132,0.126,0.035,0.027,1,2019-10-27 12:30:00,0,0,0,0,0
3617,Sardegna,0.234,0.132,0.126,0.035,0.027,1,2019-10-27 15:00:00,0,0,0,0,1
3618,Sardegna,0.234,0.132,0.126,0.035,0.027,1,2019-10-27 18:00:00,0,0,0.25,0.25,0


In [24]:
def kpi_partita(data, juv : float, inter : float, mil : float, rm : float, na : float, g_j : float, 
                g_i : float, g_m : float, g_r : float, g_n : float):
    data_cal=data[:10]
    '''
    return (juv*g_j*get_posizione_classifica(data_cal, "Juventus")
            + inter*g_i*get_posizione_classifica(data_cal, "Inter") 
            + mil*g_m*get_posizione_classifica(data_cal, "Milan") 
            + rm*g_r*get_posizione_classifica(data_cal, "Roma") 
            + na*g_n*get_posizione_classifica(data_cal, "Napoli"))#*match_hour(data)
    '''
    m_j = juv*g_j if g_j!=0 else juv*0.25 if (g_i!=0) | (g_m!=0) | (g_r!=0) | (g_n!=0) else 0
    i_i = inter*g_i if g_i!=0 else inter*0.25 if (g_j!=0) | (g_m!=0) | (g_r!=0) | (g_n!=0) else 0
    m_m = mil*g_m if g_m!=0 else mil*0.25 if (g_j!=0) | (g_i!=0) | (g_r!=0) | (g_n!=0) else 0
    m_r = rm*g_r if g_r!=0 else rm*0.25 if (g_j!=0) | (g_i!=0) | (g_m!=0) | (g_n!=0) else 0
    m_n = na*g_n if g_n!=0 else na*0.25 if (g_j!=0) | (g_i!=0) | (g_m!=0) | (g_r!=0)  else 0
    return (m_j + i_i + m_m + m_r + m_n)

def match_hour(data):
    data = datetime.strptime(data, "%Y-%m-%d %H:%M:%S")
    return 1.1 if data.hour=='21' else 0.6

kpi = pd.DataFrame(columns=['REGIONE', 'DATA', 'KPI'])

for index, row in df_anag.iterrows():
    kpi = kpi.append(pd.Series([row['REGIONE'], row['DATA'], kpi_partita(row['DATA'],
                                                                         row['JUVENTUS'], 
                                                                         row['INTER'],
                                                                         row['MILAN'],
                                                                         row['ROMA'], 
                                                                         row['NAPOLI'],
                                                                         row['GAME_J'],
                                                                         row['GAME_I'],
                                                                         row['GAME_M'], 
                                                                         row['GAME_R'], 
                                                                         row['GAME_N'])]
                               , index=kpi.columns), ignore_index=True)

kpi

,REGIONE,DATA,KPI
0,Piemonte,2019-10-29 19:00:00,0
1,Piemonte,2019-10-29 21:00:00,0.12525
2,Piemonte,2019-10-30 19:00:00,0.12525
3,Piemonte,2019-10-30 21:00:00,0.1365
4,Piemonte,2019-10-31 21:00:00,0.37525
...,...,...,...
3615,Sardegna,2019-10-26 20:45:00,0
3616,Sardegna,2019-10-27 12:30:00,0
3617,Sardegna,2019-10-27 15:00:00,0.12725
3618,Sardegna,2019-10-27 18:00:00,0.357


In [10]:
from datetime import date, timedelta
kpi['DATA']=pd.to_datetime(kpi['DATA'])
kpi['DATA'] = pd.DatetimeIndex(kpi['DATA']) + timedelta(hours=1)
kpi[kpi['REGIONE']=='Abruzzo']

,REGIONE,DATA,KPI
876,Abruzzo,2019-12-14 16:00:00,0.000
877,Abruzzo,2019-12-14 19:30:00,5.600
878,Abruzzo,2019-12-14 21:45:00,0.000
879,Abruzzo,2019-12-15 13:30:00,0.000
880,Abruzzo,2019-12-15 16:00:00,64.750
...,...,...,...
944,Abruzzo,2020-02-16 13:30:00,0.000
945,Abruzzo,2020-02-16 16:00:00,8.375
946,Abruzzo,2020-02-16 19:00:00,8.800
947,Abruzzo,2020-02-16 21:45:00,9.900


In [11]:
kpi_tmp=kpi.groupby(['REGIONE',pd.Grouper(key='DATA', freq='3H'), 'KPI']).size().to_frame('size').reset_index()
kpi_tmp['DATA'] = pd.DatetimeIndex(kpi_tmp['DATA']) - timedelta(hours=3)
kpi_tmp.drop('size',axis=1,inplace=True)
kpi_tmp

,REGIONE,DATA,KPI
0,Abruzzo,2019-12-14 12:00:00,0.000
1,Abruzzo,2019-12-14 15:00:00,5.600
2,Abruzzo,2019-12-14 18:00:00,0.000
3,Abruzzo,2019-12-15 09:00:00,0.000
4,Abruzzo,2019-12-15 12:00:00,64.750
...,...,...,...
1455,Veneto,2020-02-16 09:00:00,0.000
1456,Veneto,2020-02-16 12:00:00,7.025
1457,Veneto,2020-02-16 15:00:00,9.900
1458,Veneto,2020-02-16 18:00:00,8.450


In [8]:
traf = pd.read_csv('./final_dataset_v2.csv', keep_default_na=False, na_values=['_'])
traf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178153 entries, 0 to 178152
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ID                 178153 non-null  object 
 1   PROVINCIA          178153 non-null  object 
 2   DATA               178153 non-null  object 
 3   CAPACITA           178153 non-null  int64  
 4   SOGLIA GIALLA      178153 non-null  float64
 5   SOGLIA ROSSA       178153 non-null  float64
 6   MAX_DS             178153 non-null  float64
 7   INTERVALLO_MAX_DS  178153 non-null  object 
 8   TIPO_GIORNO_DS     178153 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 12.2+ MB


In [9]:
traf.loc[traf['SOGLIA GIALLA']<traf['MAX_DS'], 'SOPRA_SOGLIA_GIALLA'] =1
traf.loc[traf['SOGLIA ROSSA']<traf['MAX_DS'], 'SOPRA_SOGLIA_ROSSA'] =1
traf.loc[traf['CAPACITA']<traf['MAX_DS'], 'SOPRA_CAPACITA'] =1
traf.fillna({'SOPRA_SOGLIA_GIALLA':0, 'SOPRA_SOGLIA_ROSSA':0, 'SOPRA_CAPACITA':0}, inplace=True)
traf.drop(['CAPACITA', 'SOGLIA GIALLA', 'SOGLIA ROSSA'], axis=1, inplace=True)
traf[traf.SOPRA_CAPACITA==1]

,ID,PROVINCIA,DATA,MAX_DS,INTERVALLO_MAX_DS,TIPO_GIORNO_DS,SOPRA_SOGLIA_GIALLA,SOPRA_SOGLIA_ROSSA,SOPRA_CAPACITA
18688,PISTITMM-400MSANOLT,PT,02-01-2020,157.3088,19:15,FER,1.0,1.0,1.0
21911,FRNZITNC-400MSANOLT,FI,03-01-2020,180.8074,21:15,FER,1.0,1.0,1.0
30819,SGVAITFE-400MSANOLT,AR,05-01-2020,158.8630,23:00,FES,1.0,1.0,1.0
37369,FRNZITNC-400MSANOLT,FI,06-01-2020,156.5453,19:45,FES,1.0,1.0,1.0
62538,FRNZITNC-400MSANOLT,FI,11-01-2020,151.2334,21:30,FES,1.0,1.0,1.0
117733,PGGIITDB-400MSANOLT,SI,22-01-2020,159.0621,21:30,FER,1.0,1.0,1.0
126452,CASSITDM-400MSANOLT,FR,23-01-2020,157.8280,15:15,FER,1.0,1.0,1.0
157206,PISTITML-400MSANOLT,PT,29-01-2020,152.5493,20:30,FER,1.0,1.0,1.0
173062,PISTITHP-400MSANOLT,PT,02-02-2020,160.8719,19:00,FES,1.0,1.0,1.0


In [14]:
traf[traf.PROVINCIA=='RM']

,ID,PROVINCIA,DATA,MAX_DS,INTERVALLO_MAX_DS,TIPO_GIORNO_DS,SOPRA_SOGLIA_GIALLA,SOPRA_SOGLIA_ROSSA,SOPRA_CAPACITA
70,TIVOITAA-100DIRETTI_IPRGARM035/100/0/0/15,RM,2019-12-30 23:00:00,151.3430,23:00,FER,0.0,0.0,0.0
71,CLLEITBU-400MSANOLT,RM,2019-12-30 19:15:00,46.3409,19:15,FER,0.0,0.0,0.0
89,CIVIITAH-101FIGLI_IP0,RM,2019-12-30 21:30:00,51.4045,21:30,FER,0.0,0.0,0.0
104,CLLEITAD-100DIRETTI_IPRGARM011/0/6/0/0,RM,2019-12-30 22:15:00,231.9737,22:15,FER,0.0,0.0,0.0
114,FRASITAF-102FIGLI_IP0,RM,2019-12-30 22:00:00,28.5536,22:00,FER,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
178125,ROMAITDL-101DIRETTI_IPRGARM054/101/0/0/6,RM,2020-02-02 15:15:00,232.6742,15:15,FES,0.0,0.0,0.0
178132,ROMAITCX-104DIRETTI_IPRGARM025/100/0/0/11,RM,2020-02-02 19:15:00,75.2038,19:15,FES,0.0,0.0,0.0
178139,ROMAITBV-101FIGLI_IP0,RM,2020-02-02 22:15:00,47.8655,22:15,FES,0.0,0.0,0.0
178140,FRASITBF-101FIGLI_IP0,RM,2020-02-02 19:45:00,25.5942,19:45,FES,0.0,0.0,0.0


In [10]:
traf.loc[:,'DATA'] = pd.to_datetime(traf.DATA + ' ' + traf.INTERVALLO_MAX_DS, format="%d-%m-%Y %H:%M")

traf_f=traf.groupby([pd.Grouper(key='DATA', freq='3H'), 'PROVINCIA', 'TIPO_GIORNO_DS']).agg({'MAX_DS': 'sum', 
                                         'SOPRA_SOGLIA_GIALLA' : 'sum',
                                         'SOPRA_SOGLIA_ROSSA' : 'sum',
                                         'SOPRA_CAPACITA' : 'sum'}).reset_index()
tmp = traf.groupby([pd.Grouper(key='DATA', freq='3H'), 'PROVINCIA', 'TIPO_GIORNO_DS']).size().to_frame('N_APPARATI').reset_index()
traf_f=pd.merge(traf_f, tmp, on=['DATA', 'PROVINCIA', 'TIPO_GIORNO_DS'], how='outer').reset_index(drop=True)

traf_f.head()

,DATA,PROVINCIA,TIPO_GIORNO_DS,MAX_DS,SOPRA_SOGLIA_GIALLA,SOPRA_SOGLIA_ROSSA,SOPRA_CAPACITA,N_APPARATI
0,2019-12-30,AG,FER,173.9960,0.0,0.0,0.0,4
1,2019-12-30,AL,FER,72.1645,0.0,0.0,0.0,1
2,2019-12-30,AN,FER,49.8329,0.0,0.0,0.0,2
3,2019-12-30,AP,FER,200.8427,0.0,0.0,0.0,3
4,2019-12-30,AQ,FER,41.8586,0.0,0.0,0.0,1


In [15]:
#Convertiamo il tipo di giorno FER o FES in 0 o 1
traf_f.replace({'TIPO_GIORNO_DS': {'FER': 0, 'FES': 1}}, inplace=True)

#Aggiungiamo la regione
traf_f=pd.merge(traf_f, prv, on=['PROVINCIA'], how='left').reset_index(drop=True)

In [16]:
traf_f

,DATA,PROVINCIA,TIPO_GIORNO_DS,MAX_DS,SOPRA_SOGLIA_GIALLA,SOPRA_SOGLIA_ROSSA,SOPRA_CAPACITA,N_APPARATI,REGIONE
0,2019-12-23 00:00:00,AG,0,17.8873,0.0,0.0,0.0,1,Sicilia
1,2019-12-23 00:00:00,AL,0,143.0735,0.0,0.0,0.0,7,Piemonte
2,2019-12-23 00:00:00,AN,0,251.1803,0.0,0.0,0.0,4,Marche
3,2019-12-23 00:00:00,AP,0,86.8396,0.0,0.0,0.0,2,Marche
4,2019-12-23 00:00:00,AR,0,190.8758,0.0,0.0,0.0,5,Toscana
...,...,...,...,...,...,...,...,...,...
17932,2020-01-26 21:00:00,VI,1,3919.7270,0.0,0.0,0.0,28,Veneto
17933,2020-01-26 21:00:00,VR,1,2547.6016,0.0,0.0,0.0,26,Veneto
17934,2020-01-26 21:00:00,VS,1,31.3531,0.0,0.0,0.0,1,Sardegna
17935,2020-01-26 21:00:00,VT,1,1237.9667,0.0,0.0,0.0,11,Lazio


In [17]:
traf_f=pd.merge(traf_f, kpi_tmp, on=['REGIONE', 'DATA'], how='left').reset_index(drop=True)

In [18]:
traf_f=traf_f.fillna(0)
traf_f[traf_f['DATA']=='2020-01-26 18:00:00']

,DATA,PROVINCIA,TIPO_GIORNO_DS,MAX_DS,SOPRA_SOGLIA_GIALLA,SOPRA_SOGLIA_ROSSA,SOPRA_CAPACITA,N_APPARATI,REGIONE,KPI
17722,2020-01-26 18:00:00,AG,1,372.0261,0.0,0.0,0.0,5,Sicilia,19.650
17723,2020-01-26 18:00:00,AL,1,161.2582,0.0,0.0,0.0,6,Piemonte,20.075
17724,2020-01-26 18:00:00,AN,1,840.8300,0.0,0.0,0.0,9,Marche,8.500
17725,2020-01-26 18:00:00,AO,1,92.3510,0.0,0.0,0.0,1,Valle d'Aosta/Vallée d'Aoste,20.075
17726,2020-01-26 18:00:00,AP,1,148.0676,0.0,0.0,0.0,4,Marche,8.500
...,...,...,...,...,...,...,...,...,...,...
17825,2020-01-26 18:00:00,VE,1,1008.3669,0.0,0.0,0.0,9,Veneto,19.625
17826,2020-01-26 18:00:00,VI,1,1056.2818,0.0,0.0,0.0,11,Veneto,19.625
17827,2020-01-26 18:00:00,VR,1,1456.6296,0.0,0.0,0.0,14,Veneto,19.625
17828,2020-01-26 18:00:00,VT,1,245.5525,0.0,0.0,0.0,5,Lazio,18.075


In [21]:
traf_f[(traf_f.KPI!=0) & (traf_f.PROVINCIA=='NA')]

,DATA,PROVINCIA,TIPO_GIORNO_DS,MAX_DS,SOPRA_SOGLIA_GIALLA,SOPRA_SOGLIA_ROSSA,SOPRA_CAPACITA,N_APPARATI,REGIONE,KPI


In [19]:
#traf_f.to_csv (r'./datasets/to_train.csv', index = None, header=True)